# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [52]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])
nltk.download('stopwords')
import re
import time
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:////home/workspace/etl.db')
df = pd.read_sql_table('etl', engine)
X = df.message.values
Y = df.iloc[:,4:].values

In [3]:
df.shape

(26180, 40)

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [6]:
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]",  " ",text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    #stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [7]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(),n_jobs=-1))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
start = time.time()
pipeline.fit(X_train, y_train)
print(time.time()- start)

174.64885354042053


In [23]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=-1))])

In [10]:
X_test.shape

(6545,)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [12]:
score = pipeline.score(X_test, y_test)
print(score)

0.2525592055


In [13]:
target_names = df.iloc[:,4:].columns.tolist()

In [14]:
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      5027
               request       0.75      0.52      0.62      1127
                 offer       0.17      0.03      0.05        36
           aid_related       0.75      0.62      0.68      2662
          medical_help       0.60      0.27      0.37       504
      medical_products       0.59      0.31      0.41       308
     search_and_rescue       0.67      0.20      0.31       167
              security       0.50      0.07      0.12       101
              military       0.58      0.34      0.43       195
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.67      0.72       446
                  food       0.81      0.65      0.72       739
               shelter       0.77      0.56      0.65       585
              clothing       0.65      0.34      0.44        89
                 money       0.54      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
import signal

from contextlib import contextmanager

import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}

def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler

@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:
    from workspace_utils import active session
    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)

In [34]:
def build_model_cv():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(),n_jobs=-1))
    ])

    parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)), tested already and default are the best
        #'vect__max_df': (0.5, 0.75, 1.0), tested already and default are the best
        'vect__max_features': (None, 5000, 10000),
        #'tfidf__use_idf': (True, False)
        'clf__estimator__n_estimators': [50,100,200]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters,verbose = 5, n_jobs=1, scoring='f1_samples')

    return cv

In [27]:
with active_session():
    model = build_model_cv()
    model.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__n_estimators=50, vect__max_features=None ........


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=50, vect__max_features=None, score=0.5209953962918464, total= 2.4min
[CV] clf__estimator__n_estimators=50, vect__max_features=None ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=50, vect__max_features=None, score=0.5357542539929266, total= 2.4min
[CV] clf__estimator__n_estimators=50, vect__max_features=None ........


...

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=200, vect__max_features=10000, score=0.5119626154179926, total= 5.2min
[CV] clf__estimator__n_estimators=200, vect__max_features=10000 ......


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=200, vect__max_features=10000, score=0.5048024828822354, total= 5.2min


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 127.4min finished


In [33]:
pd.DataFrame(model.cv_results_)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__n_estimators,param_vect__max_features,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,116.405014,0.387301,30.430023,0.163460,50,None,"{'clf__estimator__n_estimators': 50, 'vect__ma...",0.520995,0.535754,0.528576,0.528442,0.006026,2,0.544247,0.547309,0.545919,0.545825,0.001252
1,106.396212,0.355691,30.319679,0.177889,50,5000,"{'clf__estimator__n_estimators': 50, 'vect__ma...",0.517992,0.538551,0.526699,0.527748,0.008426,3,0.542133,0.548943,0.544625,0.545234,0.002814
2,110.962285,0.400767,30.410518,0.088227,50,10000,"{'clf__estimator__n_estimators': 50, 'vect__ma...",0.522170,0.540855,0.527721,0.530249,0.007835,1,0.543736,0.551176,0.544547,0.546486,0.003333
3,174.902011,0.717038,32.305090,0.254543,100,None,"{'clf__estimator__n_estimators': 100, 'vect__m...",0.515032,0.529151,0.520431,0.521538,0.005817,5,0.557797,0.558090,0.558307,0.558065,0.000209
4,155.443511,0.171946,32.102441,0.113161,100,5000,"{'clf__estimator__n_estimators': 100, 'vect__m...",0.514824,0.524993,0.521631,0.520483,0.004230,6,0.555419,0.555276,0.554674,0.555123,0.000323
5,166.066311,0.686912,32.566855,0.068241,100,10000,"{'clf__estimator__n_estimators': 100, 'vect__m...",0.518206,0.530376,0.523238,0.523940,0.004993,4,0.558452,0.560663,0.557684,0.558933,0.001263
6,297.082436,1.893543,36.481552,0.012960,200,None,"{'clf__estimator__n_estimators': 200, 'vect__m...",0.505594,0.512267,0.505792,0.507884,0.003100,7,0.584411,0.580302,0.577161,0.580625,0.002969
7,255.944388,1.488898,35.993521,0.128769,200,5000,"{'clf__estimator__n_estimators': 200, 'vect__m...",0.504415,0.506685,0.508320,0.506474,0.001601,9,0.581395,0.571732,0.576497,0.576541,0.003945
8,274.063305,1.636113,36.228714,0.129957,200,10000,"{'clf__estimator__n_estimators': 200, 'vect__m...",0.506378,0.511963,0.504802,0.507714,0.003072,8,0.581843,0.579405,0.578799,0.580016,0.001315


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, max_features=10000)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(),n_jobs=-1))
    ])

In [38]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...ator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=-1))])

In [39]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [41]:
score = pipeline.score(X_test, y_test)
print(score)

0.251336898396


In [42]:
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      5027
               request       0.76      0.53      0.63      1127
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.64      0.68      2662
          medical_help       0.60      0.27      0.37       504
      medical_products       0.63      0.33      0.43       308
     search_and_rescue       0.62      0.19      0.29       167
              security       0.40      0.08      0.13       101
              military       0.61      0.33      0.43       195
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.65      0.70       446
                  food       0.82      0.65      0.73       739
               shelter       0.78      0.55      0.64       585
              clothing       0.69      0.37      0.48        89
                 money       0.55      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
#try with random forest
pipeline_rf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(),n_jobs=-1))
    ])

In [16]:
# train classifier
start = time.time()
pipeline_rf.fit(X_train, y_train)
print(time.time()- start)

144.5210943222046


In [17]:
# predict on test data
y_pred_rf = pipeline_rf.predict(X_test)

In [18]:
score_rf = pipeline_rf.score(X_test, y_test)
print(score_rf)

0.236363636364


In [19]:
print(classification_report(y_test, y_pred_rf, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.84      0.93      0.88      5006
               request       0.81      0.40      0.53      1149
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.60      0.67      2728
          medical_help       0.56      0.12      0.20       494
      medical_products       0.68      0.07      0.13       361
     search_and_rescue       0.33      0.02      0.04       198
              security       0.25      0.01      0.02       118
              military       0.67      0.09      0.16       227
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.41      0.55       434
                  food       0.82      0.51      0.63       744
               shelter       0.74      0.26      0.39       590
              clothing       0.69      0.08      0.15       107
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


With RandomForest the f1 score is lower, so we keep AdaBoost.

### 9. Export your model as a pickle file

In [59]:
import pickle
# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipeline, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.